# PuLPモジュールを用いた輸送問題の最適解の計算
## Pandasを使用しない場合
## PuLPモジュール読み込み

In [8]:
import pulp

## 目的関数の係数および制約条件左辺の係数を定義
送出元と受取先を各々リストとして定義する．送出元はS1とS2なので，送出元の番号を並べたリストS = [1,2]で表す．
リストは要素を並べて[]で囲んで表す．リストにおける要素へのアクセスは要素の並び順であるインデックスにより行う．
先頭の要素のインデックスは0であるから，先頭の要素へのアクセスはS[0]，2番目の要素へのアクセスはS[1]により行う．
同様に受取先D1,D2,D3をまとめてD = [1,2,3]で表す．

Cは輸送コストで，枝をキー，コストを値とする辞書で表す．
Cのキーは(1,1), (1,2),...となっているが，要素を並べて()で囲んだデータ型はタプルという．
タプルはリストに似ているが，一旦定義したタプルの要素に代入ができない，辞書のキーにすることができるという点でリストとは異なる．

Supplyは送出元の送出可能量で，送出元の番号をキーとする辞書で表す．
Demandは受取先の受取量で，受取先の番号をキーとする辞書で表す．

In [3]:
S = [1,2] # 送出元の番号
D = [1,2,3] # 受取先の番号
C = {(1,1):10, (1,2):6, (1,3):16, (2,1):8, (2,2):8, (2,3):10} # 輸送コスト
Supply = {1:100, 2:180} # 送出元の送出可能量
Demand = {1:120, 2:40, 3:80} # 受取先の受取量

## 決定変数の定義
(送出元,受取先)をキーとする辞書として定義する．SとDの要素の組み合わせに対して変数を定義するため，以下のように for による繰り返しを2重にする．

In [4]:
x ={(i,j):pulp.LpVariable(f'x{i}_{j}',0) for i in S for j in D}
x

{(1, 1): x1_1,
 (1, 2): x1_2,
 (1, 3): x1_3,
 (2, 1): x2_1,
 (2, 2): x2_2,
 (2, 3): x2_3}

## 問題の定義
各決定変数に輸送コストを乗じたものの和として目的関数を定義している．決定変数はSとDの要素の組み合わせに対応して定義されているので，総和を求めるにはlpSum()内で for による繰り返しを2重にする．

各送出元の送出可能量制約を設定する．for文で繰り返すことにより，送出元の数が増えたときには効率的な記述になる．
同様に，各受取先の受取量を設定する．

In [7]:
p = pulp.LpProblem('輸送問題', sense=pulp.LpMinimize)
p += pulp.lpSum(C[(i,j)]*x[(i,j)] for i in S for j in D), '目的関数　輸送コスト'

# 各送出元の送出可能量制約
for i in S:
    p += pulp.lpSum(x[(i,j)] for j in D) <= Supply[i], f'送出元{i}の送出可能量制約'
    
# 各受取先の受取量制約
for j in D:
    p += pulp.lpSum(x[(i,j)] for i in S) == Demand[j] , f'受取先{j}の受取量制約'
p

輸送問題:
MINIMIZE
10*x1_1 + 6*x1_2 + 16*x1_3 + 8*x2_1 + 8*x2_2 + 10*x2_3 + 0
SUBJECT TO
送出元1の送出可能量制約: x1_1 + x1_2 + x1_3 <= 100

送出元2の送出可能量制約: x2_1 + x2_2 + x2_3 <= 180

受取先1の受取量制約: x1_1 + x2_1 = 120

受取先2の受取量制約: x1_2 + x2_2 = 40

受取先3の受取量制約: x1_3 + x2_3 = 80

VARIABLES
x1_1 Continuous
x1_2 Continuous
x1_3 Continuous
x2_1 Continuous
x2_2 Continuous
x2_3 Continuous

## 最適解の計算と結果の読み取り

In [7]:
result = p.solve()

In [8]:
pulp.LpStatus[result]

'Optimal'

In [9]:
pulp.value(p.objective)

2040.0

In [14]:
for v in p.variables():
    print(f'{v} = {pulp.value(v)}')

x1_1 = 20.0
x1_2 = 40.0
x1_3 = 0.0
x2_1 = 100.0
x2_2 = 0.0
x2_3 = 80.0


S1からD1に20トン，S1からD2に40トン，S2からD1に100トン，S2からD3に80トン輸送するとき，輸送コストは最小になり，そのときのコストは2040万円となる．